**CLASSIFICATION**

Welcome dear reader. In this jupyter-notebook, you will meet the classification of nothing less but glass. Have you ever wondered about how a computer would classify your bedroom window? Wonder no more!

In [30]:
# imports
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from toolbox_02450 import rocplot, confmatplot
import pandas as pd
from sklearn import model_selection
from tqdm import tqdm
import torch

font_size = 15
plt.rcParams.update({'font.size': font_size})

In [31]:
# loading data
filename = 'glass.csv'
df = pd.read_csv(filename)

# defining x and y for the linear regression
#       all attributes except RI
X = np.array(df.iloc[:,1:10])
#       number of attributes
M = 9
#       type
y = np.array(df.iloc[:,10])

In [42]:
# the format of C needs to be this, so it is usable with torch, see before and after:
print(y)
y[y>3] = y[y>3] - 1
y = y - 1
print(y)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [45]:
# K = K_outer = K_inner
K = 10
CV_outer = model_selection.KFold(n_splits=K,shuffle=True)

#* INIT FOR LOGISTIC REGRESSION
# init for lambda-inner-loop
lambda_interval = np.logspace(-8, 2, 50)
train_error_rate = np.empty((K,len(lambda_interval)))
test_error_rate = np.empty((K,len(lambda_interval)))
# init for error save
log_train_error = np.zeros(10)
log_test_error = np.zeros(10)

opt_lambda = np.zeros(10)
#coefficient_norm = np.zeros(len(lambda_interval))


#* INIT FOR BASE-LINE
base_line_train_err = np.zeros(10)
base_line_test_err = np.zeros(10)

#* INIT FOR ANN-CLASSIFICATION
max_iter = 10000
tolerance = 1e-6
logging_frequency = 1000
best_final_loss = 1e100
n_replicates = 1
ANN_error_test = np.zeros(K)
ANN_error_train = np.zeros(K)

h = [i for i in range(1,11)]

C = len(np.unique(y))

opt_h = np.zeros(10)




#* OUTER LOOP
for m, (train_index, test_index) in tqdm(enumerate(CV_outer.split(X))):
    X_train, X_test = X[train_index, :], X[test_index, :]
    y_train, y_test = y[train_index], y[test_index]


    # TODO: CHECK IF THIS STANDARDIZATION IS CORRECT.
    mu = np.mean(X_train, axis=0)
    sigma = np.std(X_train, axis = 0)

    X_train = (X_train - mu)/sigma
    X_test = (X_test - mu)/sigma

    #* INNER LOOP
    ## ## GOAL: finding best lambda & finding best number of hidden layers
    CV_inner = model_selection.KFold(n_splits=K,shuffle=True)
    for j, (train_index_inner, test_index_inner) in enumerate(CV_inner.split(X_train, y_train)):
        # printing
        print("outer_loop: " + str(m+1) + "/10. inner_loop: " + str(j+1) + "/10.",end='\r')

        X_train_inner, X_test_inner = X_train[train_index_inner, :], X_train[test_index_inner, :]
        y_train_inner, y_test_inner = y_train[train_index_inner], y_train[test_index_inner]

        #init for optimal h
        h_error_train = np.zeros(K)
        h_error_test = np.zeros(K)

        ##########################
        #* Finding the best lambda:
        # allows us to iterate over a arbitray number of lambdas
        for k in range(0, len(lambda_interval)):
            #'newton-cg' gives better results
            mdl = LogisticRegression(penalty='l2', solver='newton-cg', C=1/lambda_interval[k], max_iter=100)
            # but the other is faster but outputs hella lot of warnings:
            # mdl = LogisticRegression(penalty='l2', C=1/lambda_interval[k], max_iter=100)
            
            mdl.fit(X_train_inner, y_train_inner)

            y_train_est_inner = mdl.predict(X_train_inner).T
            y_test_est_inner = mdl.predict(X_test_inner).T
            
            train_error_rate[j,k] = np.sum(y_train_est_inner != y_train_inner) / len(y_train_inner)
            test_error_rate[j,k] = np.sum(y_test_est_inner != y_test_inner) / len(y_test_inner)

            # føler ikke det her skal bruges
            #w_est = mdl.coef_[0] 
            #coefficient_norm[k] = np.sqrt(np.sum(w_est**2))


        #* Finding the optimal amount of hidden layers:
        X_train_inner, X_test_inner = torch.tensor(X_train_inner, dtype=torch.float), torch.tensor(X_test_inner, dtype=torch.float)
        y_train_inner, y_test_inner = torch.tensor(y_train_inner), torch.tensor(y_test_inner)
        # reshaping helps torch
        #y_test_inner = torch.reshape(y_test_inner, (y_test_inner.shape[0],1))
        #y_train_inner = torch.reshape(y_train_inner, (y_train_inner.shape[0],1))

        n_hidden_units_inner = h[j]
        model_inner = lambda: torch.nn.Sequential(
                    torch.nn.Linear(M, n_hidden_units_inner), #M features to H hiden units
                    # 1st transfer function, either Tanh or ReLU:
                    #torch.nn.Tanh(),                            
                    torch.nn.ReLU(),
                    torch.nn.Linear(n_hidden_units_inner, C),
                    torch.nn.Softmax(dim=1)) #softmax as we want classes

        # loss function for multinomial classification
        loss_fn = torch.nn.CrossEntropyLoss()

        for r in range(n_replicates):
            net_inner = model_inner()

            torch.nn.init.xavier_uniform_(net_inner[0].weight)
            torch.nn.init.xavier_uniform_(net_inner[2].weight)

            optimizer = torch.optim.Adam(net_inner.parameters())

            learning_curve = []
            old_loss = 1e6
            for i in range(max_iter):
                y_est = net_inner(X_train_inner)
                #y_class = torch.max(y_est, dim=1)[1]
                loss = loss_fn(y_est, y_train_inner)
                loss_value = loss.data.numpy()
                learning_curve.append(loss_value)


                p_delta_loss = np.abs(loss_value - old_loss)/old_loss
                if p_delta_loss < tolerance: break
                old_loss = loss_value


                #if (i != 0) & ((i+1) % logging_frequency == 0):
                #    print_str = '\t\t' + str(i+1) + '\t' + str(loss_value) + '\t' + str(p_delta_loss)
                #    print(print_str)
                optimizer.zero_grad(); loss.backward(); optimizer.step()

    

            if loss_value < best_final_loss:
                best_net = net_inner
                best_final_loss = loss_value
                best_learning_curve = learning_curve

        y_pred_inner = net_inner(X_test_inner)
        
        softmax_logits = net_inner(torch.tensor(X_test, dtype=torch.float))
        y_test_est = (torch.max(softmax_logits, dim=1)[1]).data.numpy() 
        h_error_test[j]  = np.sum(y_test_est != np.array(y_test))/len(y_test)


         
        #h_error_train[j] = best_final_loss

    #* Optimal values found from inner loop
    #  the smallesst of the means of the differnet lambdas
    opt_lambda[m] = lambda_interval[np.argmin(np.mean(test_error_rate,axis=0))]
    opt_h[m] = h[np.argmin(h_error_test)]


    #* CLASSIFICATION
    ##########################
    ## BASELINE
    # find out which class has the highest count
    big_class = np.argmax([np.count_nonzero(y_train == i) for i in range(1,7)])
    # assign everything to this class (calculate error rate)
    base_line_train_err[m] = np.sum(y_train != big_class)/len(y_train)
    base_line_test_err[m] = np.sum(y_test != big_class)/len(y_test)


    ##########################
    ## Logistic Regression
    log = LogisticRegression(penalty='l2', C=1/opt_lambda[m], max_iter=1000)
    log.fit(X_train, y_train)
    y_train_est = log.predict(X_train)
    y_test_est = log.predict(X_test)

    log_train_error[m] = np.sum(y_train_est != y_train)/len(y_train)
    log_test_error[m] = np.sum(y_test_est != y_test)/len(y_test)

    ##########################
    ## ANN-Classification

    model = lambda: torch.nn.Sequential(
                    torch.nn.Linear(M, int(opt_h[m])), #M features to H hiden units
                    # 1st transfer function, either Tanh or ReLU:
                    #torch.nn.Tanh(),                            
                    torch.nn.ReLU(),
                    torch.nn.Linear(int(opt_h[m]), C), # H hidden units to 1 output neuron
                    torch.nn.Softmax(dim=1) # final tranfer function
                    )


    X_train, y_train = torch.tensor(X_train, dtype = torch.float), torch.tensor(y_train)
    X_test, y_test = torch.tensor(X_test, dtype = torch.float), torch.tensor(y_test)
    n_replicates = 3
    for r in range(n_replicates):
        net = model()

        torch.nn.init.xavier_uniform_(net[0].weight)
        torch.nn.init.xavier_uniform_(net[2].weight)

        optimizer = torch.optim.Adam(net.parameters())

        learning_curve = []
        old_loss = 1e6
        for i in range(max_iter):
            y_est = net(X_train)
            #y_class = torch.max(y_est, dim=1)[1]
            loss = loss_fn(y_est, y_train)
            loss_value = loss.data.numpy()
            learning_curve.append(loss_value)


            p_delta_loss = np.abs(loss_value - old_loss)/old_loss
            if p_delta_loss < tolerance: break
            old_loss = loss_value


            if (i != 0) & ((i+1) % logging_frequency == 0):
                #print_str = '\t\t' + str(i+1) + '\t' + str(loss_value) + '\t' + str(p_delta_loss)
                ###print(print_str)
            optimizer.zero_grad(); loss.backward(); optimizer.step()

        

        #print('\t\tFinal Loss:')
        #print_str = '\t\t' + str(i+1) + '\t' + str(loss_value) + '\t' + str(p_delta_loss)
        ##print(print_str)

        if loss_value < best_final_loss:
            best_net = net
            best_final_loss = loss_value
            best_learning_curve = learning_curve
        
    
    softmax_logits = net(torch.tensor(X_test, dtype=torch.float))
    y_test_est = (torch.max(softmax_logits, dim=1)[1]).data.numpy() 
    ANN_error_test[m] = np.sum(y_test_est != np.array(y_test))/len(y_test)




# WARNING: Inden du kligger på den flotte trekant.
# Det tager cirka 20 min

0it [00:00, ?it/s]

/Users/nikolaj/opt/anaconda3/envs/AndSem/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


		1000	1.6041638	4.3099346e-05
		2000	1.5512775	2.5973233e-05
		3000	1.5186186	1.5620959e-05
		4000	1.4994612	1.0414577e-05
		5000	1.4876357	6.250365e-06
		6000	1.4801749	4.0268487e-06
		7000	1.4748989	3.0713556e-06
		8000	1.4707879	2.3504826e-06
		9000	1.4672636	2.3561283e-06
		10000	1.4624377	2.6084442e-06
		Final Loss:
		10000	1.4624377	2.6084442e-06
		1000	1.6727899	4.254265e-05
		2000	1.5942508	3.364739e-05
		3000	1.5545429	2.3081491e-05
		4000	1.5235143	1.572725e-05
		5000	1.5082074	7.824937e-06
		6000	1.49869	6.204263e-06
		Final Loss:
		6715	1.4935346	7.1835154e-07
		1000	1.599934	4.410731e-05
		2000	1.5615548	1.3588357e-05
		3000	1.5418086	1.3066542e-05
		4000	1.5131378	2.2058708e-05
		5000	1.4895768	1.6885826e-05
		6000	1.4617945	1.0030545e-05
		7000	1.4516252	5.4199736e-06
		8000	1.4445194	4.6213913e-06
		9000	1.4387635	3.977042e-06


/var/folders/k7/4m0jgf1s5rggf6q996sfswf00000gn/T/ipykernel_47516/1964271695.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  softmax_logits = net(torch.tensor(X_test, dtype=torch.float))
1it [01:18, 78.79s/it]

		10000	1.4307209	6.832286e-06
		Final Loss:
		10000	1.4307209	6.832286e-06
		1000	1.5772481	3.3934517e-0510/10.
		2000	1.5484796	9.007133e-06
		3000	1.5393186	5.4984157e-06
		4000	1.5317684	3.7355676e-06
		5000	1.5233885	6.494933e-06
		6000	1.5138925	4.8033344e-06
		7000	1.5079952	3.082999e-06
		8000	1.5039372	2.5364673e-06
		9000	1.5005602	2.2244044e-06
		10000	1.4975656	1.6716402e-06
		Final Loss:
		10000	1.4975656	1.6716402e-06
		1000	1.592652	5.373909e-05
		2000	1.5394487	2.8573197e-05
		3000	1.5092288	8.767468e-06
		4000	1.4988106	5.2493433e-06
		5000	1.4914703	5.1952516e-06
		6000	1.4848961	3.2915198e-06
		7000	1.4808583	2.1734988e-06
		8000	1.4777822	1.936021e-06
		9000	1.4752358	1.4545228e-06
		10000	1.4732205	1.2137607e-06
		Final Loss:
		10000	1.4732205	1.2137607e-06
		1000	1.6597729	0.0001445581
		2000	1.537581	3.3414493e-05
		3000	1.5040683	1.529654e-05
		4000	1.4866844	8.980602e-06
		5000	1.472609	8.74264e-06
		6000	1.4599401	7.430418e-06
		7000	1.450051	5.9191148e-06
		8

...

		Final Loss:
		8552	1.4398788	7.451202e-07
		1000	1.6049416	9.87036e-05: 10/10.
		2000	1.5230931	3.0210495e-05
		3000	1.4896922	1.6244358e-05
		4000	1.4710997	1.0210197e-05
		Final Loss:
		4538	1.4639736	8.957151e-07
		1000	1.5946923	4.6868427e-05
		2000	1.5474498	1.8796422e-05
		3000	1.5299034	6.3114385e-06
		4000	1.5231841	3.130519e-06
		Final Loss:
		4660	1.5203425	4.7045677e-07
		1000	1.5798529	0.00011943249
		2000	1.4656572	4.099118e-05
		3000	1.427107	1.5369676e-05
		4000	1.4108068	8.280662e-06
		5000	1.4017015	5.442923e-06
		6000	1.3949963	3.8454564e-06
		7000	1.390671	2.4858941e-06
		8000	1.3877113	1.7180676e-06


/var/folders/k7/4m0jgf1s5rggf6q996sfswf00000gn/T/ipykernel_47516/1964271695.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  softmax_logits = net(torch.tensor(X_test, dtype=torch.float))
9it [22:13, 151.36s/it]

		9000	1.3855221	1.3766262e-06
		Final Loss:
		9427	1.3847538	9.46956e-07
		1000	1.5494926	9.16974e-05p: 10/10.
		2000	1.4752246	3.547326e-05
		3000	1.4469836	1.202802e-05
		4000	1.4335507	7.1514214e-06
		5000	1.4208446	5.5373894e-06
		6000	1.4144167	3.7083764e-06
		7000	1.4100341	2.7899296e-06
		8000	1.4066143	2.118723e-06
		9000	1.4040706	1.6980497e-06
		Final Loss:
		9456	1.4031864	9.3451655e-07
		1000	1.5658559	5.3288466e-05
		2000	1.5091842	2.5196927e-05
		3000	1.4796327	1.46629445e-05
		4000	1.462137	8.560667e-06
		5000	1.4497613	8.633744e-06
		6000	1.4399672	5.8777796e-06
		7000	1.4316818	5.7452667e-06
		Final Loss:
		7993	1.4243736	8.369242e-08
		1000	1.6033188	3.6877023e-05
		2000	1.5565553	2.3128232e-05
		3000	1.5301028	1.20758e-05
		4000	1.5126386	8.274859e-06
		5000	1.5023817	5.4749044e-06
		6000	1.4955722	3.8259764e-06
		7000	1.4906843	3.0388321e-06
		8000	1.4867767	2.485564e-06


/var/folders/k7/4m0jgf1s5rggf6q996sfswf00000gn/T/ipykernel_47516/1964271695.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  softmax_logits = net(torch.tensor(X_test, dtype=torch.float))
10it [24:31, 147.11s/it]

		9000	1.4831392	2.4112842e-06
		Final Loss:
		9238	1.4723316	5.6676464e-07


In [46]:
print("{:<15} | {:<15} {:<15} {:<15} {:<15} {:<15}".format('Outer fold i', 'h', 'E_ANN', 'lambda', 'E_log', 'E_base'))
print("-----------------------------------------------------------------------------------------")
for i in range(10):
    print("{:<15} | {:<15} {:<15.3f} {:<15.6f} {:<15.3f} {:<15.7f}".format(i+1, int(opt_h[i]), ANN_error_test[i], opt_lambda[i], log_test_error[i], base_line_test_err[i]))


Outer fold i    | h               E_ANN           lambda          E_log           E_base         
-----------------------------------------------------------------------------------------
1               | 1               0.364           0.000000        0.227           0.6363636      
2               | 1               0.318           0.086851        0.273           0.6818182      
3               | 1               0.409           0.138950        0.409           0.5909091      
4               | 1               0.364           0.005179        0.318           0.5000000      
5               | 1               0.667           1.456348        0.381           0.7619048      
6               | 1               0.571           0.910298        0.524           0.6666667      
7               | 1               0.476           0.568987        0.286           0.6666667      
8               | 1               0.429           0.054287        0.571           0.6190476      
9               | 1         